# Advanced Analysis & Statistics

* Take RCS (Radar Cross Section) data and use it to calculate "Collision Probability".
* Start looking at Correlation Matrices to see if an object's age (Launch Year) is a strong predictor of its tracking health.